###  LINKS

In [1]:
player_uri = 'https://www.euroleaguebasketball.net/euroleague/stats/expanded/?size=1000&viewType=traditional&seasonCode=E2022&statisticMode=accumulated&seasonMode=Single&sortDirection=ascending&statistic='
real_madrid = 'https://www.euroleaguebasketball.net/euroleague/teams/real-madrid/statistics/mad/?season=2022-23&phase=All%20phases'
teams = 'https://www.euroleaguebasketball.net/euroleague/teams'
game_center = 'https://www.euroleaguebasketball.net/euroleague/game-center/'
standings = 'https://www.euroleaguebasketball.net/euroleague/standings/'
driver_path  = r'driver/chromedriver.exe'

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time
from urllib.parse import urljoin

### GET PLAYER DATA

In [3]:
standings_uri = 'https://www.euroleaguebasketball.net/euroleague/standings/'
player_stats_uri = 'https://www.euroleaguebasketball.net/euroleague/stats/expanded/?size=1000&viewType=traditional&seasonCode=E2022&statisticMode=accumulated&seasonMode=Single&sortDirection=ascending&statistic='
player_section_uri = 'https://www.euroleaguebasketball.net/euroleague/players/'
home = 'https://www.euroleaguebasketball.net'
driver_path = r'driver/chromedriver.exe'

In [4]:
class Extractor:
    def __init__(self, driver_path, standings_uri, player_stats_uri, player_section_uri):
        service = Service(driver_path)
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        self.driver = webdriver.Chrome(service=service, options=options)
        self.standings_uri = standings_uri
        self.player_stats_uri = player_stats_uri
        self.player_section_uri = player_section_uri        
        
    def extract_player_data(self):
        self.driver.get(self.player_stats_uri)
        time.sleep(5)
        soup = BeautifulSoup(self.driver.page_source,'lxml')
        table = soup.find(name='div', attrs='complex-stat-table_body__yaXeJ')
        rows = table.find_all(name='div', attrs='complex-stat-table_row__1P6us')

        header_list = []
        for cell in rows[0]:
            header_list.append(cell.text)

        data = []
        for row in rows:
            row_data = []
            for cell in row.find_all('div', {'role':'cell'}):
                if cell.find('a'):
                    span = cell.find('span', attrs={'class':'complex-stat-table_longValue__3xUTT'})
                    row_data.append(span.text)
                else:
                    row_data.append(cell.text)
            data.append(row_data)
        return pd.DataFrame(data[1:], columns=header_list)
    
    def extract_team_data(self):
        self.driver.get('https://www.euroleaguebasketball.net/euroleague/stats/key-stats-teams/?size=200&misc=&category=Score&seasonMode=All')
        time.sleep(3)
        button = self.driver.find_element(by=By.CLASS_NAME, value='settings-button_button__1ij1C')
        self.driver.execute_script("arguments[0].click();", button)
        time.sleep(2)
        soup = BeautifulSoup(self.driver.page_source,'lxml')
        cl = soup.find('select', attrs={'class':'text-input_wrapper__1R8vF'})
        soup = BeautifulSoup(extractor.driver.page_source,'lxml')
        categories = []
        for option in cl.find_all('option'):
            categories.append(option['value'])
        df = pd.DataFrame()
        for category in categories:
            url = f'https://www.euroleaguebasketball.net/euroleague/stats/key-stats-teams/?size=200&misc=&seasonCode=E2022&category={category}&seasonMode=Single&sortDirection=descending&aggregate=accumulated'
            extractor.driver.get(url)
            time.sleep(3)
            soup = BeautifulSoup(extractor.driver.page_source, 'lxml')
            table = soup.find(name='div', attrs='complex-stat-table_body__yaXeJ')
            rows = table.find_all(name='div', attrs='complex-stat-table_row__1P6us')

            header_list = []
            for cell in rows[0].find_all('div', attrs={'class':'complex-stat-table__head__3wslB'}):
                span = cell.find('span')
                if span:
                    header_list.append(span.text.replace('*','').strip())

            data = []
            for row in rows:
                row_data = []
                for cell in row.find_all('div', {'role':'cell'}):
                    if cell.find('a'):
                        span = cell.find('span', attrs={'class':'complex-stat-table_longValue__3xUTT'})
                        row_data.append(span.text)
                    else:
                        row_data.append(cell.text)
                data.append(row_data)
            if df.empty:
                df = pd.DataFrame(data[1:], columns=header_list)
            else:        
                df2 = pd.DataFrame(data[1:], columns=header_list)
                cols_to_drop = [col for col in df2.columns if col in df.columns and col != 'Team']
                df = df.merge(df2.drop(cols_to_drop, axis=1), on='Team', how='inner')
        return df
        

    def extract_team_df(self):
        self.driver.get(self.standings_uri)
        time.sleep(5)
        soup = BeautifulSoup(self.driver.page_source,'lxml')
        table = soup.find(name='div', attrs='complex-stat-table_body__yaXeJ')
        rows = table.find_all(name='div', attrs='complex-stat-table_row__1P6us')

        header_list = []
        for cell in rows[0].find_all('div', attrs={'class':'complex-stat-table_cell__1lxC7'}):
            span = cell.find('span', attrs={'class':'complex-stat-table_tabletHeadName__1Jn7u'})
            if span:
                header_list.append(span.text.replace('*','').strip())
            else:
                header_list.append(cell.find('span').text.replace('*','').strip())

        data = []
        for row in rows:
            row_data = []
            for cell in row.find_all('div', {'role':'cell'}):
                if cell.find('a'):
                    span = cell.find('span', attrs={'class':'complex-stat-table_mainClubName__3IMZJ'})
                    row_data.append(span.text)
                else:
                    row_data.append(cell.text)
            data.append(row_data)
        return pd.DataFrame(data[1:], columns=header_list)
    
    def get_player_links(self):
        self.driver.get(self.player_section_uri)
        
        #Expand buttons
        load_more_class = 'all-players_loadMoreBtn__35J6M'
        load_more_button = self.driver.find_element(by="class name", value=load_more_class)
        while load_more_button: 
#             load_more_button.click()
            self.driver.execute_script("arguments[0].click();", load_more_button)
            time.sleep(2)
            try:
                load_more_button = self.driver.find_element(by="class name", value=load_more_class)
            except NoSuchElementException:
                print('all players expanded')
                load_more_button = None
                
        #Extract URI
        soup = BeautifulSoup(self.driver.page_source,'lxml')
        player_list = soup.find('ul', attrs={'class':'all-players-list_filterList__1jH_E'})
        home = 'https://www.euroleaguebasketball.net'
        urls = []
        for li in player_list.find_all('li'):
            path = urljoin(home, li.find('a')['href'])
            urls.append(path)
        print(f'Num player links: {len(urls)}')
        return urls

    def player_info(self, player_info_uri):
        self.driver.get(player_info_uri)
        time.sleep(3)
        
        info={}
        soup = BeautifulSoup(self.driver.page_source,'lxml')
        spans = soup.find('h1', attrs={'class':'hero-info_fullName__IDuVE'}).find_all('span')
        name = ''
        for span in spans:
            name += f' {span.text}'
        name = name.strip()
        info['name'] = name
        info['team'] = soup.find('h2', attrs={'class':'hero-info_teamName__3g98M'}).text
        for li in soup.find('ul', attrs={'class','hero-info_dataList__2BmgP'}):
            try:
                info[li.find('span').text] = li.find('b').text
            except AttributeError:
                info['blank'] = 'blank'
                print('Blank Data')
        try:
            img = soup.find('div', attrs={'class':'player-hero_imageWrap__161BQ cover-contain_ofCover__3K_IS'}).find('img')
            info['pic_link'] = img['data-srcset'].split('?')[0]
        except AttributeError:
            info['pic_link'] = None
        return info
    
    def player_info_df(self, start, end):
        uri_list = self.get_player_links()
        players = []
        for uri in uri_list[start:end]:
            print(uri)
            players.append(self.player_info(uri))
        return pd.DataFrame(players)

In [5]:
extractor = Extractor(driver_path, standings_uri, player_stats_uri, player_section_uri)

In [9]:
team_stats = extractor.extract_team_data()
team_stats.to_csv('team_stats.csv', index=False)

In [10]:
player_stats = extractor.extract_player_data()
player_stats.to_csv('player_stats.csv', index = False)

In [11]:
team_standings = extractor.extract_team_df()
team_standings.to_csv('team_standings.csv', index = False)

In [16]:
player_info = extractor.player_info_df(200,350)
player_info.to_csv('player_info3.csv', index = False)

all players expanded
Num player links: 301
https://www.euroleaguebasketball.net/euroleague/players/amine-noua/006068/
https://www.euroleaguebasketball.net/euroleague/players/james-nunnally/007400/
https://www.euroleaguebasketball.net/euroleague/players/retin-obasohan/011938/
https://www.euroleaguebasketball.net/euroleague/players/andreas-obst/004508/
https://www.euroleaguebasketball.net/euroleague/players/semi-ojeleye/011982/
https://www.euroleaguebasketball.net/euroleague/players/elie-okobo/011212/
https://www.euroleaguebasketball.net/euroleague/players/louis-olinde/006919/
https://www.euroleaguebasketball.net/euroleague/players/yakuba-ouattara/004197/
https://www.euroleaguebasketball.net/euroleague/players/alessandro-pajola/006739/
https://www.euroleaguebasketball.net/euroleague/players/kevin-pangos/006433/
https://www.euroleaguebasketball.net/euroleague/players/georgios-papagiannis/005844/
https://www.euroleaguebasketball.net/euroleague/players/kostas-papanikolaou/001255/
https://ww

In [6]:
import os
os.listdir()

['.git',
 '.gitignore',
 '.ipynb_checkpoints',
 'data_extraction_practice.ipynb',
 'driver',
 'geckodriver.log',
 'player_info.csv',
 'player_info2.csv',
 'player_info3.csv',
 'player_stats.csv',
 'README.md',
 'team_standings.csv',
 'team_stats.csv',
 'venv']

In [7]:
df1 = pd.read_csv('player_info.csv')
df2 = pd.read_csv('player_info2.csv')
df3 = pd.read_csv('player_info3.csv')

In [11]:
df1

,name,team,Nationality,Height,Born,pic_link,blank
0,ALBERTO ABALDE,Real Madrid,Spain,202.0,15 Dec 1995,https://media-cdn.incrowdsports.com/82c96970-c...,NaN
1,AWUDU ABASS,Virtus Segafredo Bologna,Italy,198.0,27 Jan 1993,https://media-cdn.incrowdsports.com/41895699-9...,NaN
2,VENIAMIN ABOSI,Olympiacos Piraeus,Greece,194.0,15 Jan 2006,NaN,NaN
3,ALEX ABRINES,FC Barcelona,Spain,198.0,1 Aug 1993,https://media-cdn.incrowdsports.com/e37b7de4-d...,NaN
4,JALEN ADAMS,Maccabi Playtika Tel Aviv,United States of America,191.0,11 Dec 1995,https://media-cdn.incrowdsports.com/14bb2746-d...,NaN
...,...,...,...,...,...,...,...
296,GUERSCHON YABUSELE,Real Madrid,France,204.0,17 Dec 1995,https://media-cdn.incrowdsports.com/ab7df3d6-8...,NaN
297,PAUL ZIPSER,FC Bayern Munich,Germany,203.0,18 Feb 1994,https://media-cdn.incrowdsports.com/b49a4766-4...,NaN
298,IFTAH ZIV,Maccabi Playtika Tel Aviv,Israel,191.0,9 Jul 1995,https://media-cdn.incrowdsports.com/39750d46-0...,NaN
299,ANTE ZIZIC,Anadolu Efes Istanbul,Croatia,210.0,4 Jan 1997,https://media-cdn.incrowdsports.com/c9621cbf-2...,NaN
